In [4]:
# LLM dynamic evaluation

import replicate
import pandas as pd
import json
import os
from config import config
from dotenv import load_dotenv
load_dotenv()
import time
import random
from functools import wraps

folder_path = 'files'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

INSTRUCTION = config.INSTRUCTION
F_NAME = config.F_NAME
config.set_mode("dynamic")

def load_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def get_random_perturbation(perturbations):
    category = random.choice(list(perturbations.keys()))
    perturbation = random.choice(list(perturbations[category].items()))
    return category, perturbation

# # Load the file
df = pd.read_excel(config.questions)
# Save the original DataFrame
df.to_excel(config.q_original, index=False)

# Trim whitespace and newline characters
df['Question'] = df['Question'].str.strip()  # Removes leading/trailing whitespace

# Check for duplicate questions
duplicates = df.duplicated(subset=['Question'], keep=False)
if duplicates.any():
    print("Duplicates found. Removing duplicates.")

    # Remove duplicates, keeping the first occurrence
    df = df.drop_duplicates(subset=['Question'], keep='first')

    # Save the modified DataFrame, overwriting the original questions file
    df.to_excel(config.q_original, index=False)
else:
    print("No duplicates found.")

perturbations = load_file(config.perturbations)
knowledgebase = load_file(config.knowledgebase)

# Modify DataFrame to include new columns
results_df = pd.DataFrame(columns=['Model', 'Question', 'Response', 'Perturbed Question', 'Perturbed Response', 'Final Analysis Question', 'Final Analysis Response', 'Latency', 'Category', 'Type'])

models = {
    # "qwen": "lucataco/qwen1.5-110b:af7953cb4fe4948df44a074d4785c2f74d0096257197198e90c9ac84361b6aa9",
    "llama3": "meta/meta-llama-3-70b-instruct",
    "yi": "01-ai/yi-34b-chat:914692bbe8a8e2b91a4e44203e70d170c9c5ccc1359b283c84b0ec8d47819a46",
    # "mistral-7b": "mistralai/mistral-7b-instruct-v0.2:f5701ad84de5715051cb99d550539719f8a7fbcf65e0e62a3d1eb3f94720764e",
    "noushermes": "nousresearch/hermes-2-theta-llama-8b",
    "mixtral": "mistralai/mixtral-8x7b-instruct-v0.1",
    # "deepseek_33bq": "kcaverly/deepseek-coder-33b-instruct-gguf:ea964345066a8868e43aca432f314822660b72e29cab6b4b904b779014fe58fd",
    }

def generate_prompt(model_key, instruction, question):
    # prompt_for_qwen = "system\n {instruction}. Please try your best to answer the following question. \nuser\n{question}\nassistant\n"
    # prompt_for_hermes = """[
    # {{
    #   "role": "system",
    #   "content": "{instruction}. Please try your best to answer the following question." 
    # }},
    # {{
    #   "role": "user",
    #   "content": {question}
    # }}
    # ]"""

    # if model_key in ["yi-34b", "qwen-110b"]:
    #     return prompt_for_qwen.format(instruction=instruction, question=question)
    # elif model_key == "noushermes2":
    #     return prompt_for_hermes.format(instruction=instruction, question=question)
    # else:
    #     # plain_text_question = json.loads(question)
    #     return f"{instruction}. Please try your best to answer the following question. {question}"

    return f"{instruction}. Please try your best to answer the following question. {question}"

prompt_for_qwen="""<|im_start|>system\n {INSTRUCTION}. Please try your best to answer the following question. <|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"""
prompt_for_hermes = """[
{{
  "role": "system",
  "content": "{INSTRUCTION}. Please try your best to answer the following question." 
}},
{{
  "role": "user",
  "content": {question}
}}
]"""
def retry_on_exception(max_retries=3, exceptions=(Exception,)):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            retries = 0
            while retries < max_retries:
                try:
                    return func(*args, **kwargs)
                except exceptions as e:
                    retries += 1
                    print(f"Retrying {func.__name__} due to {e}, attempt {retries}/{max_retries}")
            raise Exception(f"Failed after {max_retries} retries")
        return wrapper
    return decorator

@retry_on_exception(max_retries=3, exceptions=(replicate.exceptions.ModelError,))
def ask_llm(model_value, prompt):   
    output = replicate.run(
        model_value,
        input={
            "debug": False,
        #   "top_k": 50,
            "top_p": 0.9,
            "prompt": prompt,
            "temperature": 0.7,
            "max_new_tokens": 500,
            "min_new_tokens": -1
        }
    )
    response = ""
    for item in output:
        item_str = str(item)  # Convert item to string
        response += item_str if len(item_str) == 1 else f"{item_str}"
    response = response.strip()
    return response

# Iterate through each model
for model_key, model_value in models.items():
    responses = []

    for index, row in df.iterrows():
        start_time = time.time()  # Record the start time
        qn = row['Question']
        Category = row['Category']
        Type = row['Type']
        question = json.dumps(qn)
        prompt = generate_prompt(model_key, config.INSTRUCTION, question)
        try:
            print(f"{model_key}: {prompt}")
            response = ask_llm(model_value, prompt)
            
        except Exception as e:
            response = f"Error: {e}"

        # Perturb the question and get the response
        category, (perturbation, description) = get_random_perturbation(perturbations)
        perturbed_qn = f"{qn}\nResponse: {response}\nChange in circumstances: {perturbation} - {description}\n What should change in the response?"
        prompt = generate_prompt(model_key, config.INSTRUCTION, perturbed_qn)
        # print(f"{model_key}: Perturbed Question: {prompt}")
        perturbed_response = ask_llm(model_value, prompt)

        # Evaluate sufficiency or suggest alternate course
        final_analysis_qn = f"Original Question: {question}\nOriginal Response: {response} \nPerturbation ({category}): {perturbation} - {description}\n {perturbed_response}\nKnowledgebase: {knowledgebase}\nNow consider the knowlegebase, what else ought we to do?"
        prompt = generate_prompt(model_key, config.INSTRUCTION, final_analysis_qn)
        # print(f"{model_key}: Final Analysis Question: {prompt}")
        final_analysis_response = ask_llm(model_value, prompt)

        end_time = time.time()  # Record the end time
        latency = (end_time - start_time)/3  # Calculate average latency

        # Record each stage
        new_row = {
            'Model': model_key, 
            'Question': qn, 
            'Response': response, 
            'Perturbed Question': perturbed_qn, 
            'Perturbed Response': perturbed_response, 
            'Final Analysis Question': final_analysis_qn, 
            'Final Analysis Response': final_analysis_response,
            'Latency': latency,
            'Category': Category,
            'Type': Type
        }
        results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)

        if index % 10 == 0:
            results_df.to_excel(config.llmresults_file_path, index=False, sheet_name='Sheet1')
            
results_df.to_excel(config.llmresults_file_path, index=False, sheet_name='Sheet1')

No duplicates found.
llama3: You are a brilliant and conscientious AI assistant working in an enterprise.. Please try your best to answer the following question. "Are pathogenicity scores of mutations or copy number profiles associated with cancer progression and survival for the gene POLR3E?"


/var/folders/bl/nkj7dq8s6m16hf4zmr47vkfr0000gn/T/ipykernel_54549/233625127.py:185: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)


llama3: You are a brilliant and conscientious AI assistant working in an enterprise.. Please try your best to answer the following question. "Are there any first in class opportunities for the gene POLR3E?"
llama3: You are a brilliant and conscientious AI assistant working in an enterprise.. Please try your best to answer the following question. "Describe a strategy to screen for synergistic effects between drug candidates."
llama3: You are a brilliant and conscientious AI assistant working in an enterprise.. Please try your best to answer the following question. "Describe associations between expression and variant profiles for the gene POLR3E."
llama3: You are a brilliant and conscientious AI assistant working in an enterprise.. Please try your best to answer the following question. "Describe the process of building a chemogenomics model to predict drug-target interactions."
llama3: You are a brilliant and conscientious AI assistant working in an enterprise.. Please try your best to 

In [7]:
# GPT-4 dynamic evaluation

import pandas as pd
import json
from config import config
import openai
import requests
from openai import OpenAI
import time
from dotenv import load_dotenv
load_dotenv()
import os
import random

INSTRUCTION = config.INSTRUCTION
F_NAME = config.F_NAME
config.set_mode("dynamic")

folder_path = 'files'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

def load_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def get_random_perturbation(perturbations):
    category = random.choice(list(perturbations.keys()))
    perturbation = random.choice(list(perturbations[category].items()))
    return category, perturbation

perturbations = load_file(config.perturbations)
knowledgebase = load_file(config.knowledgebase)

client = OpenAI()
def show_json(obj):
    print(json.loads(obj.model_dump_json()))

assistant = client.beta.assistants.create(
    name=f"{F_NAME} AI Dynamic Evaluator",
    instructions=config.INSTRUCTION,
    model=config.GPT_MODEL,
)
show_json(assistant)

# Utility functions
def read_csv(file_path):
    return pd.read_excel(file_path)

def process_data_for_gpt(data):
    prompts = []
    for _, row in data.iterrows():
        question = row['Question']
        prompt = f"Please try your best to answer the following question.:\n\n{question}"
        prompts.append(prompt)
    return prompts

def submit_message_and_create_run(assistant_id, prompt):
    thread = client.beta.threads.create() # If you replace this globally it appends all answers to the one before.
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=prompt)
    return client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id), thread

def wait_on_run_and_get_response(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(0.5)
    messages = client.beta.threads.messages.list(thread_id=thread.id, order="asc")
    return [m.content[0].text.value for m in messages if m.role == 'assistant']

def ask_gpt4(prompt, ASSISTANT_ID):
    run, thread = submit_message_and_create_run(ASSISTANT_ID, prompt)
    response = wait_on_run_and_get_response(run, thread)
    if isinstance(response, list):
        response = ' '.join(map(str, response))
    response = response.replace("\\\\n", "\\n")
    response = response.strip()
    print(response)
    return response

def process_question_with_gpt4(row, assistant_id):
    start_time = time.time()  # Capture start time
    original_question = row['Question']
    category = row.get('Category', 'Static')  # Default to 'Static' if not present
    if category != "Dynamic":
        # Dynamic question processing logic
        first_response = ask_gpt4(original_question, assistant_id)
        category, (perturbation, description) = get_random_perturbation(perturbations)
        perturbed_qn = f"{original_question}\nResponse: {first_response}\nChange in circumstances: {perturbation} - {description}\n What should change in the response?"
        perturbed_response = ask_gpt4(perturbed_qn, assistant_id)
        final_analysis_qn = f"Original Question: {original_question}\nOrig Response: {first_response} \nPerturbation ({category}): {perturbation} - {description}\n {perturbed_response}\nKnowledgebase Content: {knowledgebase}\n Now consider the knowlegebase, what else ought we to do?"
        final_analysis_response = ask_gpt4(final_analysis_qn, assistant_id)
    else:
        # Static question processing logic
        first_response = ask_gpt4(original_question, assistant_id)
        perturbed_qn = perturbed_response = final_analysis_qn = final_analysis_response = "n/a"

    end_time = time.time()  # Capture end time
    latency = (end_time - start_time)/3  # Calculate latency

    return {
        'Model': config.GPT_MODEL,
        'Question': original_question, 
        'Response': first_response, 
        'Perturbed Question': perturbed_qn, 
        'Perturbed Response': perturbed_response, 
        'Final Analysis Question': final_analysis_qn, 
        'Final Analysis Response': final_analysis_response,
        'Latency': latency,
        'Category': row['Category'],
        'Type': row['Type']
    }

# Modify DataFrame to include new columns
new_data_columns = ['Model', 'Question', 'Response', 'Perturbed Question', 'Perturbed Response', 'Final Analysis Question', 'Final Analysis Response', 'Latency', 'Category', 'Type']
results_df = pd.DataFrame(columns=new_data_columns)
data = read_csv(config.questions)
prompts = process_data_for_gpt(data)
ASSISTANT_ID = assistant.id

# Process each question
for index, row in data.iterrows():
    processed_info = process_question_with_gpt4(row, ASSISTANT_ID)
    results_df = pd.concat([results_df, pd.DataFrame([processed_info])], ignore_index=True)

# Save the results
results_df.to_excel(config.gpt4results_csv_path, index=False)


{'id': 'asst_fFDcCTTbpeC1LFKdS98VhssB', 'created_at': 1716509791, 'description': None, 'instructions': 'You are a brilliant and conscientious AI assistant working in an enterprise.', 'metadata': {}, 'model': 'gpt-4o', 'name': 'loop evals AI Dynamic Evaluator', 'object': 'assistant', 'tools': [], 'response_format': 'auto', 'temperature': 1.0, 'tool_resources': {'code_interpreter': None, 'file_search': None}, 'top_p': 1.0}
The gene POLR3E, encoding a subunit of RNA polymerase III, has been studied in the context of cancer, but it is not among the most commonly known oncogenes or tumor suppressor genes. For detailed insights into the association of specific mutations or copy number variations (CNVs) in POLR3E with cancer progression and patient survival, both experimental studies and bioinformatics analyses are necessary.

**Pathogenicity Scores of Mutations:**
Pathogenicity scores, such as those from tools like PolyPhen-2, SIFT, or MutationTaster, predict the potential impact of amino ac

/var/folders/bl/nkj7dq8s6m16hf4zmr47vkfr0000gn/T/ipykernel_54549/1980936076.py:123: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([processed_info])], ignore_index=True)


POLR3E encodes a subunit of RNA polymerase III, which is involved in the transcription of small RNAs, such as tRNAs and 5S rRNA. As of the latest data up to 2023, POLR3E and related components of the RNA polymerase III complex have not been widely targeted in drug development, suggesting potential first-in-class opportunities for innovative therapeutics.

Here are a few areas where first-in-class opportunities might exist for targeting POLR3E:

1. **Cancer Treatment**: RNA polymerase III is involved in cell growth and proliferation. Inhibition of POLR3E could theoretically lead to reduced cancer cell proliferation, providing a novel target for cancer therapies.

2. **Autoimmune Diseases**: Since POLR3E is involved in the fundamental process of transcription, modulating its activity could have implications in autoimmune diseases, where the immune system attacks the body's own cells. Targeting POLR3E might help modulate immune responses.

3. **Infectious Diseases**: Some viruses hijack t